# Initial setup

In [1]:
import boto3
import sagemaker
import pandas as pd
try:
    from pyathena import connect
except:
    !pip install --disable-pip-version-check -q PyAthena==2.1.0
    from pyathena import connect

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

bucket = 'ads508team7'
raw_file_dir   = f"s3://{bucket}/raw_files"
s3_staging_dir = f"s3://{bucket}/athena/staging"
database_name  = 'sys'

# Create Database in Athena

In [2]:
# Create connection to the staging directory.
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [3]:
# Create the database in athena
query_create_database = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
pd.read_sql(query_create_database, conn)

# Verify if database is created.
df_show = pd.read_sql("SHOW DATABASES", conn)
df_show

,database_name
0,default
1,sys


### Create aisles table in Athena

In [4]:
table_name = 'aisles'

create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS
    {database_name}.{table_name}
        (
        aisle_id     int ,
        aisle        string
        )
         
    ROW FORMAT DELIMITED 
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    LOCATION '{raw_file_dir}/{table_name}'
    TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

# verify if table is created successfully
pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [5]:
table_name = 'departments'

create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS
    {database_name}.{table_name}
        (
        department_id     int ,
        department        string
        )
         
    ROW FORMAT DELIMITED 
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    LOCATION '{raw_file_dir}/{table_name}'
    TBLPROPERTIES ('skip.header.line.count'='1')
"""


pd.read_sql(create_table, conn)

# verify if table is created successfully
pd.read_sql(f'SELECT * FROM {database_name}.{table_name}  LIMIT 5', conn)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


### Create orders table in Athena

In [6]:
table_name = 'orders'

create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS
    {database_name}.{table_name}
        (
order_id                     int,
user_id                      int,
eval_set                     string,
order_number                 int,
order_dow                    int,
order_hour_of_day            int,
days_since_prior_order       int

        )
         
    ROW FORMAT DELIMITED 
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    LOCATION '{raw_file_dir}/{table_name}'
    TBLPROPERTIES ('skip.header.line.count'='1')
"""


pd.read_sql(create_table, conn)

# verify if table is created successfully
pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,691886,128019,prior,15,1,11,14
1,834999,128019,prior,16,1,14,14
2,3289584,128019,prior,17,4,20,10
3,3345985,128019,prior,18,1,11,11
4,1851406,128019,test,19,6,11,30


### Create products table in Athena

In [7]:
table_name = 'products'

create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS
    {database_name}.{table_name}
        (
        product_id      int,
        product_name    string,
        aisle_id        int,
        department_id   int
        )
         
    ROW FORMAT DELIMITED 
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    LOCATION '{raw_file_dir}/{table_name}'
    TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

# verify if table is created successfully
pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


### Create order_products__train table in Athena

In [8]:
table_name = 'order_products__train'

create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS
    {database_name}.{table_name}
        (
        order_id          bigint,
        product_id        bigint,
        add_to_cart_order int,
        reordered         int
        )     
    ROW FORMAT DELIMITED 
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    LOCATION '{raw_file_dir}/{table_name}'
    TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

# verify if table is created successfully
pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


### Create order_products__prior table in Athena

In [9]:
table_name = 'order_products__prior'

create_table = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS
    {database_name}.{table_name}
        (
        order_id          bigint,
        product_id        bigint,
        add_to_cart_order int,
        reordered         int
        )     
    ROW FORMAT DELIMITED 
    FIELDS TERMINATED BY ','
    STORED AS TEXTFILE
    LOCATION '{raw_file_dir}/{table_name}'
    TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(create_table, conn)

# verify if table is created successfully
pd.read_sql(f'SELECT * FROM {database_name}.{table_name} LIMIT 5', conn)

,order_id,product_id,add_to_cart_order,reordered
0,1038322,16521,5,1
1,1038323,30696,1,1
2,1038323,9405,2,1
3,1038323,12322,3,1
4,1038323,19863,4,1


## Release Resources

In [10]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>